##part c) use pure tensorflow tensor datastructure to build the same - do not use the auto differentiation yet.

---



---






In [1]:
import numpy as np
import random
from numpy import array
from numpy.random import uniform
from numpy import hstack
import tensorflow as tf


In [2]:

n, d = 400, 3
num_hidden1=5 # number of neurons for first hidden layer
num_hidden2=4 # numbe rof neurons for second hidden layer
x = tf.random.uniform(minval=-1, maxval=1, shape = (n, d))
x2 = tf.cast(x**2, tf.float32)
weights_true = tf.cast(tf.constant([[5,1,1],[4,1,1]]),tf.float32)
bias_true = tf.cast(tf.constant([1,2]), tf.float32)
y_true = tf.matmul(x2, weights_true, transpose_b=True) + tf.matmul(x,weights_true,transpose_b=True) + bias_true

print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')



x: (400, 3), weights: (2, 3), bias: (2,), y: (400, 2)


In [3]:
class MSE:
  def __call__(self, y_pred, y_true):
    self.y_pred = y_pred
    self.y_true = y_true
    return tf.reduce_mean((y_pred - y_true) ** 2)

  def backward(self):
    n = self.y_true.shape[0]
    self.gradient = 2. * (self.y_pred - self.y_true) / n
    return self.gradient

In [4]:
class Linear:
  def __init__(self, input_dim: int, num_hidden: int = 1):
    self.weights = tf.Variable(tf.random.uniform(shape=(input_dim, num_hidden)))
    self.bias = tf.Variable(tf.zeros(shape=(num_hidden,)))
  
  def __call__(self, x):
    self.x = x
    return tf.matmul(x, self.weights) + self.bias

  def backward(self, gradient):
    self.weights_gradient = tf.matmul(self.x,gradient,transpose_a=True)
    self.bias_gradient = tf.reduce_sum(gradient, 0)
    self.x_gradient = tf.matmul(gradient, self.weights, transpose_b=True)
    return self.x_gradient

  def update(self, lr):
    self.weights = self.weights - lr * self.weights_gradient
    self.bias = self.bias - lr * self.bias_gradient

In [5]:
##Relu activation function
class Relu:
    def __call__(self, input_):
      import math
      self.input_ = input_
      self.output = tf.clip_by_value(self.input_, 0, math.inf)
      return self.output
    
    def backward(self, output_gradient):
      self.input_gradient = tf.cast((self.input_ > 0), tf.float32) * output_gradient
      return self.input_gradient

In [6]:
from typing import Callable

class Model:
  def __init__(self, input_dim, num_hidden):
    self.linear1 = Linear(input_dim, num_hidden)
    self.relu1 = Relu()
    self.linear2 = Linear(num_hidden,12) # Pick 12 as number of neurons in hidden layer
    self.relu2 = Relu()
    self.linear3 = Linear(12,2)
  
  # Forward pass
  def __call__(self, x):
    l1 = self.linear1(x)
    r1 = self.relu1(l1)
    l2 = self.linear2(r1)
    r2 = self.relu2(l2)
    l3 = self.linear3(r2)
    return l3
  
  def backward(self, output_gradient):
    linear3_gradient = self.linear3.backward(output_gradient)
    relu2_gradient = self.relu2.backward(linear3_gradient)
    linear2_gradient = self.linear2.backward(relu2_gradient)
    relu1_gradient = self.relu1.backward(linear2_gradient)
    linear1_gradient = self.linear1.backward(relu1_gradient)
    return linear1_gradient

  def update(self, lr):
    self.linear3.update(lr)
    self.linear2.update(lr)
    self.linear1.update(lr)

In [7]:
# Training
def fit(x, y, model: Callable, loss: Callable, lr: float, num_epochs: int):
  for epoch in range(num_epochs):
    y_pred = model(x)
    loss_value = loss(y_pred, y)
    if epoch % 50 == 0:
      print(f'Epoch {epoch}, loss {loss_value}')
    gradient_from_loss = loss.backward()
    model.backward(gradient_from_loss)
    model.update(lr)

loss = MSE()
model = Model(d, 20)
fit(x, y_true, model=model, loss=loss, lr=0.0025, num_epochs=1000)

Epoch 0, loss 229.04019165039062
Epoch 50, loss 2.0700695514678955
Epoch 100, loss 0.7154936194419861
Epoch 150, loss 0.4927928149700165
Epoch 200, loss 0.38093551993370056
Epoch 250, loss 0.30756109952926636
Epoch 300, loss 0.2553243935108185
Epoch 350, loss 0.21605010330677032
Epoch 400, loss 0.18775150179862976
Epoch 450, loss 0.16739830374717712
Epoch 500, loss 0.15188375115394592
Epoch 550, loss 0.13972370326519012
Epoch 600, loss 0.12985707819461823
Epoch 650, loss 0.12153387814760208
Epoch 700, loss 0.114504873752594
Epoch 750, loss 0.10849609971046448
Epoch 800, loss 0.10315950214862823
Epoch 850, loss 0.0983213409781456
Epoch 900, loss 0.09416122734546661
Epoch 950, loss 0.0902637466788292


In [8]:
def plot_intereactive_3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

In [10]:
x

<tf.Tensor: shape=(400, 3), dtype=float32, numpy=
array([[ 0.6458638 , -0.12703013, -0.6487    ],
       [ 0.633476  ,  0.26356268,  0.47194314],
       [-0.58075714,  0.45600677,  0.20688343],
       ...,
       [-0.7977314 , -0.0358007 , -0.10205173],
       [ 0.72341514,  0.36551976,  0.5882962 ],
       [ 0.62591624, -0.79728293,  0.2534759 ]], dtype=float32)>

In [9]:
from sklearn.manifold import TSNE
X_red = TSNE(n_components=2).fit_transform(x)
y_true_red = TSNE(n_components=1).fit_transform(y_true)
y_pred_red = TSNE(n_components=1).fit_transform(model(x)) #not using the .detach function
print(f'X_red: {X_red.shape}, y_true_red: {y_true_red.shape}, y_pred_red: {y_pred_red.shape}')
plot_intereactive_3d(X_red,y_true_red,y_pred_red)

X_red: (400, 2), y_true_red: (400, 1), y_pred_red: (400, 1)
